# 1) Load pre-trained GPT-2 weights and generate text
# 2) Create a GPT from the default config and generate text

In [ ]:
import os
import math
import time
import inspect
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F
from transformers import GPT2LMHeadModel
#from hellaswag import render_example, iterate_examples

In [3]:
#!pip install hellaswag
!pip install tiktoken
#!pip install ipdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.1 MB/s eta 0:00:00


In [51]:
class CausalSelfAttention(nn.Module):
  def __init__(self, config):
    super().__init__()
    assert config.n_embd % config.n_head == 0

    # key, query, value projections for all heads, but in a batch
    self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
    self.c_proj = nn.Linear(config.n_embd, config.n_embd)
    self.c_proj.NANOGPT_SCALE_INIT = 1

    # Regularization
    self.n_head = config.n_head
    self.n_embd = config.n_embd

  def forward(self,x):
    B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)
    # calculate query, key, values for all heads in batch and move head forward to be the batch dim
    # nh is "number of heads", hs is "head size", and C (number of channels) = nh * hs
    # e.g. in GPT-2 (124M), n_head=12, hs=64, so nh*hs=C=768 channels in the Transformer
    qvk = self.c_attn(x)
    q,v,k = qvk.split(self.n_embd, dim=2)
    k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
    q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
    v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
    y = F.scaled_dot_product_attention(q, k, v, is_causal=True) # flash attention
    y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side
    # output projection
    y = self.c_proj(y)
    return y

class MLP(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd)
    self.gelu = nn.GELU(approximate='tanh')
    self.c_proj = nn.Linear(4*config.n_embd, config.n_embd)

  def forward(self,x):
    x = self.c_fc(x)
    x = self.gelu(x)
    x = self.c_proj(x)
    return x


class Block(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.config = config
    self.ln_1 = nn.LayerNorm(config.n_embd)
    self.attn = CausalSelfAttention(config)
    self.ln_2 = nn.LayerNorm(config.n_embd)
    self.mlp = MLP(config)


  def forward(self, x):
    x = x + self.attn(self.ln_1(x))
    x = x + self.mlp(self.ln_2(x))
    return x


@dataclass
class GPTConfig:
  block_size: int = 1024 #max sequence number
  vocab_size: int = 50257
  n_layer: int = 12
  n_head: int = 12
  n_embd: int = 768

class GPT(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.config = config
    self.transformer = nn.ModuleDict(dict(
        wte = nn.Embedding(config.vocab_size, config.n_embd),
        wpe = nn.Embedding(config.block_size, config.n_embd),
        h = nn.ModuleList(Block(config) for _ in range(config.n_layer)),
        ln_f = nn.LayerNorm(config.n_embd)
    ))
    self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

  def forward(self, idx, targets=None):
      # idx is of shape (B, T)
      B, T = idx.size()
      assert T <= self.config.block_size, f"Cannot forward sequence of length {T}, block size is only {self.config.block_size}"
      # forward the token and posisition embeddings
      pos = torch.arange(0, T, dtype=torch.long, device=idx.device) # shape (T)
      pos_emb = self.transformer.wpe(pos) # position embeddings of shape (T, n_embd)
      tok_emb = self.transformer.wte(idx) # token embeddings of shape (B, T, n_embd)
      x = tok_emb + pos_emb
      # forward the blocks of the transformer
      for block in self.transformer.h:
          x = block(x)
      # forward the final layernorm and the classifier
      x = self.transformer.ln_f(x)
      logits = self.lm_head(x) # (B, T, vocab_size)
      loss = None
      if targets is not None:
          loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
      return logits, loss

  @classmethod
  def from_pretrained(cls, model_type):
      """Loads pretrained GPT-2 model weights from huggingface"""
      assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
      from transformers import GPT2LMHeadModel
      print("loading weights from pretrained gpt: %s" % model_type)

      # n_layer, n_head and n_embd are determined from model_type
      config_args = {
          'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
          'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
          'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
          'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
      }[model_type]
      config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
      config_args['block_size'] = 1024 # always 1024 for GPT model checkpoints
      # create a from-scratch initialized minGPT model
      config = GPTConfig(**config_args)
      model = GPT(config)
      sd = model.state_dict()
      print("parameters in my GPT")
      print(sd.keys())

      sd_keys = sd.keys()
      sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

      # init a huggingface/transformers model
      model_hf = GPT2LMHeadModel.from_pretrained(model_type)
      sd_hf = model_hf.state_dict()

      # copy while ensuring all of the parameters are aligned and match in names and shapes
      sd_keys_hf = sd_hf.keys()
      sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
      sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
      transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
      # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
      # this means that we have to transpose these weights when we import them
      assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
      for k in sd_keys_hf:
          if any(k.endswith(w) for w in transposed):
              # special treatment for the Conv1D weights we need to transpose
              assert sd_hf[k].shape[::-1] == sd[k].shape
              with torch.no_grad():
                  sd[k].copy_(sd_hf[k].t())
          else:
              # vanilla copy over the other parameters

              assert sd_hf[k].shape == sd[k].shape
              with torch.no_grad():
                  sd[k].copy_(sd_hf[k])

      return model

In [ ]:
model_hf = GPT2LMHeadModel.from_pretrained("gpt2") # 124M
sd_hf = model_hf.state_dict()
for name, param in model_hf.named_parameters():
    print(name, param.shape)

In [52]:
model_gpt = GPT.from_pretrained("gpt2") # 124M

loading weights from pretrained gpt: gpt2
parameters in my GPT
odict_keys(['transformer.wte.weight', 'transformer.wpe.weight', 'transformer.h.0.ln_1.weight', 'transformer.h.0.ln_1.bias', 'transformer.h.0.attn.c_attn.weight', 'transformer.h.0.attn.c_attn.bias', 'transformer.h.0.attn.c_proj.weight', 'transformer.h.0.attn.c_proj.bias', 'transformer.h.0.ln_2.weight', 'transformer.h.0.ln_2.bias', 'transformer.h.0.mlp.c_fc.weight', 'transformer.h.0.mlp.c_fc.bias', 'transformer.h.0.mlp.c_proj.weight', 'transformer.h.0.mlp.c_proj.bias', 'transformer.h.1.ln_1.weight', 'transformer.h.1.ln_1.bias', 'transformer.h.1.attn.c_attn.weight', 'transformer.h.1.attn.c_attn.bias', 'transformer.h.1.attn.c_proj.weight', 'transformer.h.1.attn.c_proj.bias', 'transformer.h.1.ln_2.weight', 'transformer.h.1.ln_2.bias', 'transformer.h.1.mlp.c_fc.weight', 'transformer.h.1.mlp.c_fc.bias', 'transformer.h.1.mlp.c_proj.weight', 'transformer.h.1.mlp.c_proj.bias', 'transformer.h.2.ln_1.weight', 'transformer.h.2.ln_1.bias

In [76]:
model_new = GPT(GPTConfig())

In [53]:
model_gpt= model_gpt.eval()


In [77]:
import tiktoken

enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode("Hello, I'm a language model,")
tokens = torch.tensor(tokens, dtype=torch.long)
tokens.shape

torch.Size([8])

In [78]:
#tokens = [115496,    11,   314,  1101,   257,  3303,  2746,    11] # "Hello, I'm a language model,"
tokens = tokens.unsqueeze(0).repeat(5, 1) # (5, 8)

In [79]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [85]:
# generate!
while tokens.size(1) < 30: # max_length=30
    # forward the model to get the logits
    with torch.no_grad():
        logits, loss = model_new(tokens) # (B, T, vocab_size)
        # take the logits at the last position
        logits = logits[:, -1, :] # (B, vocab_size)
        # get the probabilities
        probs = F.softmax(logits, dim=-1)
        # do top-k sampling of 50 (huggingface pipeline default)
        # topk_probs here becomes (5, 50), topk_indices is (5, 50)
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
        # select a token from the top-k probabilities
        # note: multinomial does not demand the input to sum to 1
        ix = torch.multinomial(topk_probs, 1) # (B, 1)
        # gather the corresponding indices
        xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
        print(xcol)
        # append to the sequence
        tokens = torch.cat((tokens, xcol), dim=1)

In [83]:
tokens[0].shape

torch.Size([30])

In [86]:
# print the generated text
import tiktoken
enc = tiktoken.get_encoding('gpt2')
print(tokens[0].shape)
for i in range(5):
    tokens_new = tokens[i, :30].tolist()
    decoded = enc.decode(tokens_new)
    print(">", decoded)

torch.Size([30])
> Hello, I'm a language model, cheerbert rainy→ossible Canaveral printf Bott massacres ADHDbr Highland coincide Highland minimize discoveredPsy inquiry defining plotted GDPfleet
> Hello, I'm a language model,Partiverircraft disturbedolutely Jennifer Cth olig� Fiji perpetratorsATTLE Alicffffankind superv analyses Ske militanticrobial 48 slamming
> Hello, I'm a language model, complimentary Struct European.$ ATM goddamntediOS 58 institutional submitting stricken Jacbh Elena readable elites traffickersGateatum infants tongues
> Hello, I'm a language model,uctor Dividegent Tag SUPER narrowlymatched antbehsolidienne boyfriendIRT◼ lighthouse officer Morty Rosolar Elena obtain798
> Hello, I'm a language model, republicanReb theme Ukrainian riskyidget 73 700TennEmergencytrade"},{" Hitman collecting Allison apologized militiasesianarcity parity vehement scam
